## Heap
Is a specialized tree-based data structure which is essentially an almost complete tree that satisfies the *heap property*:
- **max heap:** every parent node is greater than child node
- **min heap:** every child node is greater than its parent  

Heap is commonly implemented as binary tree. Heap is a complete binary tree.  

![Heap](https://i.imgur.com/lSDL2yg.png)

Binary tree heap can also be represented as an array.  

![Heap as Array](https://i.imgur.com/kuZfd6b.png)

In the array representation of a heap, for a node at index $i$
- its parent node is at $(i - 1)/2$
- its left child node is at $2i + 1$
- its right child node is at $2i + 2$  

We add a new item in $O(logn)$ worst case time and $O(1)$ best case by:
- if no space is available, resize the array
- add the new item to the end of the array
- then *heapify up* (keep comparing with parent node)  

We remove an item in $O(n)$ time by:
- find the item, get its index
- replace the item with the last item
- *heapify down*, (keep comparing left and right child)  

To search for an item, we need to traverse the whole array in $O(n)$ time

To get min or max element, $O(1)$ time.

### Implementation

In [ ]:
public class MinHeap<T extends Comparable<T>> {
    private T[] array;
    private int occupancy;

    @SuppressWarnings("unchecked")
    public MinHeap(int size) {
        array = (T[]) new Comparable[size];
    }

    public void add(T element) {
        if (occupancy == array.length) {
            throw new IllegalStateException("Heap is full");
        }

        array[occupancy] = element;
        heapifyUp(occupancy);
        occupancy++;
    }

    private void heapifyUp(int index) {
        if (index <= 0) return;

        int parentIndex = (index - 1) / 2;
        if (array[parentIndex].compareTo(array[index]) > 0) {
            swap(parentIndex, index);
            heapifyUp(parentIndex);
        }
    }

    private void swap(int indexA, int indexB) {
        T temp = array[indexA];
        array[indexA] = array[indexB];
        array[indexB] = temp;
    }

    public T remove(T element) {
        int pos = -1;
        for (int i = 0; i < occupancy; i++) {
            if (array[i].equals(element)) {
                pos = i;
                break;
            }
        }

        if (pos == -1) return null;

        T removed = array[pos];
        
        array[pos] = array[occupancy - 1];
        array[occupancy - 1] = null;
        occupancy--;

        // IMPORTANT: If we didn't remove the very last element, 
        // we might need to heapify UP or DOWN. As an example, remove
        // 6 in the heap [1, 5, 2, 7, 6, 3, 4] to see why heapifyUp is
        // required
        if (pos < occupancy) {
            heapifyDown(pos);
            heapifyUp(pos); 
        }

        return removed;
    }

    private void heapifyDown(int index) {
        int smallest = index;
        int leftChild = 2 * index + 1;
        int rightChild = 2 * index + 2;

        if (leftChild < occupancy && array[leftChild].compareTo(array[smallest]) < 0) {
            smallest = leftChild;
        }

        if (rightChild < occupancy && array[rightChild].compareTo(array[smallest]) < 0) {
            smallest = rightChild;
        }

        if (smallest != index) {
            swap(smallest, index);
            heapifyDown(smallest);
        }
    }

    public T peek() {
        return array[0];
    }

    public T poll() {
        if (occupancy == 0) {
            return null;
        }

        T temp = peek();
        remove(temp);
        return temp;
    }

    @Override
    public String toString() {
        return Arrays.toString(Arrays.copyOf(array, occupancy));
    }
}

### Building a Heap
If we insert elements one by one into a heap, the time taken to construct a tree is $O(nlogn)$ . However if we are given all the elements, it is possible to build tree in $O(n)$ time. For a given complete binary tree (which a heap is), the number of leaf nodes = $\frac{n}{2}$ if $n$ is even, else $\frac{(n+1)}{2}$.  
![Leaf Nodes](images/D7vp707.png)  

Consider the worst possible case to build a min heap, the given elements will be sorted in descending order `[7 6 5 4 3 2 1]` :   
![Build Heap](images/7l3peb0.png)

In general, a level at depth $i$ will have $2^i$ nodes. Nodes at last level will not require any swapping, nodes at level one above it would require 1 swap, 1 above that would require 2 swaps and so on.  

Total number of swaps 
$$ = 0*\frac{n}{2} + 1*\frac{n}{4} + 2*\frac{n}{8} + ...$$

This is an Arithmatic Geometric series and will lead to $O(n)$ .  

So we can say that to build a tree from given array, all the leaf nodes will not require to be heapified. So we will find all the non-leaf nodes and do the heapify operation in reverse.  

In [ ]:
public MinHeap(T[] input) {
    array = Arrays.copyOf(input, input.length);
    occupancy = array.length;
    
    int lastNonLeafNode = (array.length - 2) / 2;
    for (int i = lastNonLeafNode; i >= 0; i--) {
        heapifyDown(i);
    }
}

This will build a min heap in $O(n)$ time.

### Built-in Implementation
**Java:** Use `PriorityQueue`

In [ ]:
// Creating empty priority queue 
PriorityQueue<Integer> pQueue = new PriorityQueue<Integer>();

// Adding items to the pQueue using add() 
pQueue.add(10); 
pQueue.add(30); 
pQueue.add(20);

// Removing the top priority element (or head)
int min = pQueue.poll();

To create a max heap, just construct as:

In [ ]:
PriorityQueue<Integer> pQueue =  
             new PriorityQueue<Integer>(Collections.reverseOrder());

## Problems

**Q 1** Given are some ropes with various lengths. We can add two ropes to get one rope, the cost of this operation is the sum of the length of the ropes. Find the total minimum cost to join all ropes to form one single rope. For example, consider the ropes of lengths `2 5 2 6`. First if we add `2 + 5 = 7`, the ropes now are `7 2 6`. Then we add `7 + 2 = 9`, the ropes are `9 6`. Finally we add `9 + 6 = 15` to get the final rope. The total cost is `7+9+15 = 31`. Is this the minimum cost? If not then we need to change the order of joining ropes and get the minimum cost.  

**Answer** We need to add ropes having minimum length each time. We can make use of a min heap for this

In [3]:
import java.util.stream.IntStream;
public int minCost(int[] ropes) {
    // Build heap from collection in O(n) using constructor
    PriorityQueue<Integer> queue = 
        new PriorityQueue<>(IntStream.of(ropes).boxed().toList());

    int cost = 0;
    while (queue.size() > 1) {
        int first = queue.poll();
        int second = queue.poll();
        int sum = first + second;
        cost += sum;
        
        queue.offer(sum);
    }

    return cost;
}

int[] ropes = {2,5,2,6,3};
System.out.println(minCost(ropes));

40


[GFG](https://www.geeksforgeeks.org/problems/minimum-cost-of-ropes-1587115620/1)

**Q 2** Given $n$ distinct integers, how many min heap can be formed using those integers?  
**Answer** Out of the $n$ elements, the minimum element will be the root for all the heaps. We need to figure out in how many different ways the remaining $n-1$ itemes can be arranged. Let $l$ be the number of items in the left tree and $r$ be the number of elements in the right tree. We can say that $l + r = n - 1$. Out of the $n-1$ items, we can pick any $l$ number of items for left tree. The remaining $r$ items will be used for right tree. We can see that we get the following recurrence relation (the function $H$ represents count):
$$H(n) = ^{n-1}C_{l} \cdot H(l) \cdot H(r)$$

The next step is to find the values of $l$ and $r$. 

**Q 3** Given a list of operations and operation defined as multiplying an item of list by -1. If we repeat the operation $B$ number of times, what is the maximum sum that can be obtained?  
**Answer** We'll make use of min heap and each time pick the minimum item, negate it and put it back to the list. We repeat this B times

In [4]:
public int maxSum(int[] arr, int b) {
    PriorityQueue<Integer> queue = new PriorityQueue<>(IntStream.of(arr).boxed().toList());

    for (int i = 0; i < b; i++) {
        queue.offer(-1 * queue.poll());
    }

    return queue.stream().reduce((x, y) -> x + y).get();
}

System.out.println(maxSum(new int[]{2, -5, 10, -3, -6}, 4));

22


The running time for the above approach is $O(n + Blogn)$. There is another approach based on sorting.

In [19]:
def operation_result_sort(values, B):
    values = sorted(values)

    i = 0
    counter = 0
    while counter < B:
        if values[i % len(values)] < 0:
            values[i % len(values)] = -values[i % len(values)]
        else:
            break
        i += 1
        counter += 1

    j = i % len(values)
    if values[(i-1) % len(values)] < values[i % len(values)]:
        j = (i-1) % len(values)

    if (B-counter) % 2 != 0:
        values[j] *= -1

    return sum(values)

values = [2, -5, 10, -3, -6]
print(operation_result_sort(values, 4))

22


In the above case, we take $O(n + nlogn)$ time. So if we have $B >> n$, then this approach is better.

**Q 4** There are $n$ bags containing chocolates represented by array $A$. A kid selects a bag and eats all chocolates in it. At the same time the magician refills the bag with $A[i]/2$ number of chocolates. What is the maximum number of chocolates eaten by kid in $B$ tries?  
**Answer**

In [5]:
public int maxChocolates(int[] chocolates, int b) {
    PriorityQueue<Integer> queue = new PriorityQueue<>(Collections.reverseOrder());
    Arrays.stream(chocolates).forEach(queue::offer);

    int output = 0;
    for (int i = 0; i < b; i++) {
        int chocolate = queue.poll();
        if (chocolate == 0) break;

        output += chocolate;

        queue.offer(chocolate / 2);
    }

    return output;
}

System.out.println(maxChocolates(new int[]{2, 6, 4, 1}, 3));

13


**Q 5** Given a K-sorted array, bring it back to its original form. The original form is the sorted form. Consider the sorted array `[4, 7, 8, 9, 10, 50, 60, 70]`, then the k-sorted version will be array obtained by shifting each element by maximum of k places each side. If $k=4$, then the k-sorted version can be: `[10, 9, 8, 7, 4, 70, 60, 50]` .   
**Answer:** The easiest approach is to sort the array in $O(nlogn)$ time complexity. However we can come up with a better approach using heaps. One observation we can make is (for k=4):  
![K Sorted](images/m0OHHzD.png)  
We have range where the minimum, the next minimum, etc items will lie. So what we can do is:
- Make a heap of $k+1$ elements $O(k+1)$  
- Remove minimum element from heap $(n-k)log(k+1)$  
- Push $i+1$ element into the heap

In [7]:
public int[] kSorted(int[] input, int k) {
    PriorityQueue<Integer> queue = new PriorityQueue<>();
    for (int i = 0; i < k + 1; i++) {
        queue.offer(input[i]);
    }

    int[] output = new int[input.length];

    int i = k + 1;
    while (i < input.length) {
        output[i - k - 1] = queue.poll();
        queue.offer(input[i]);
        i++;
    }

    while (!queue.isEmpty()) {
        output[i - k - 1] = queue.poll();
        i++;
    }

    return output;
}

System.out.println(Arrays.toString(kSorted(new int[]{10, 9, 8, 7, 4, 70, 60, 50}, 4)));

[4, 7, 8, 9, 10, 50, 60, 70]


**Q 6** Sort an array without using extra space using heaps  
**Answer** The idea is to form a max heap (if we are sorting in ascending order), pop the max element and place it at the end.

In [8]:
public void sort(int[] arr) {
    // Form max heap
    int lastNonLeafIndex = (arr.length - 2) / 2;
    for (int i = lastNonLeafIndex; i >= 0; i--) {
        heapifyDown(arr, i, arr.length);
    }

    // Pop n-1 elements
    int i = 0;
    while (i < arr.length - 1) {
        int popped = arr[0];
        arr[0] = arr[arr.length - 1 - i];
        arr[arr.length - 1 - i] = popped;

        heapifyDown(arr, 0, arr.length - 1 - i);
        i++;
    }
}

private void heapifyDown(int[] arr, int index, int end) {
    int position = index;

    int leftChild = 2 * index + 1;
    if (leftChild < end && arr[position] < arr[leftChild]) {
        position = leftChild;
    }

    int rightChild = 2 * index + 2;
    if (rightChild < end && arr[position] < arr[rightChild]) {
        position = rightChild;
    }

    if (position != index) {
        int temp = arr[position];
        arr[position] = arr[index];
        arr[index] = temp;

        heapifyDown(arr, position, end);
    }
}


int[] items = {7, 6, 2, 8, 9, 5, 1};
sort(items);
System.out.println(Arrays.toString(items));

[1, 2, 5, 6, 7, 8, 9]


The above sort is not stable, because heap is not stable. But heap can be made stable. One more thing to consider is that whenever we remove items from a heap, we are reducing its size by 1. So the time taken to remove all elements would be:
$$logn + log(n-1) + log(n-2) + ... + log(1)$$
$$= log(n!)$$
And
$$log(n!) \le nLogn$$

**Q 7** There are $k$ sorted arrays each with $n$ elements, return the consolidated array containing $kn$ elements in sorted order.  
**Answer** If we consider the example $k=3, n=4$:  
```
[10 15 25 28]
[1 49 51 68]
[9 9 11 12]
```
We can see that the first column contains the lowest element, once that element is taken out (`1` in this case) we need to take into consideration only the next element (`49` in this case). So we can create a heap of `k` elements and use it to sort all elements.

In [10]:
private static record Element(int value, int row, int column) {}

public int[] knSort(int[][] arr) {
    PriorityQueue<Element> queue = new PriorityQueue<>((a, b) -> a.value() - b.value());
    for (int i = 0; i < arr.length; i++) {
        queue.offer(new Element(arr[i][0], i, 0));
    }

    int[] result = new int[arr.length * arr[0].length];
    int i = 0;
    while (!queue.isEmpty()) {
        Element temp = queue.poll();
        result[i] = temp.value();

        int row = temp.row();
        int col = temp.column();
        if (col + 1 < arr[row].length) {
            queue.offer(new Element(arr[row][col + 1], row, col + 1));
        }

        i++;
    }

    return result;
}

System.out.println(Arrays.toString(knSort(new int[][]{{10, 15, 25, 28}, {1, 49, 51, 68}, {9, 9, 11, 12}})));

[1, 9, 9, 10, 11, 12, 15, 25, 28, 49, 51, 68]


**Q 8** Given an array, find the $k$th largest element.  
**Answer** If we can maintain a min heap of $k$ elements, then the first element of the heap will be the $k$th largest.

In [11]:
public int kthLargest(int[] arr, int k) {
    // Create heap out of k elements
    PriorityQueue<Integer> queue = new PriorityQueue<>(IntStream.of(arr).limit(k).boxed().toList());

    // Iterate over rest of the elements and replace top if it is smaller
    for (int i = k; i < arr.length; i++) {
        if (arr[i] > queue.peek()) {
            queue.poll();
            queue.offer(arr[i]);
        }
    }

    return queue.peek();
}

System.out.println(kthLargest(new int[]{10, 9, 11, 2, 15, 12}, 4));

10


[LeetCode 215](https://leetcode.com/problems/kth-largest-element-in-an-array)

**Q 9** Find $k$th largest element in every range. Range here would be `[0:0], [0:1], [0:2], ... [0,end]`. If there is no $k$th largest element in a range, return-1.   
**Answer**

In [12]:
public int[] kthLargestInRange(int[] arr, int k) {
    PriorityQueue<Integer> queue = new PriorityQueue<>();

    int[] result = new int[arr.length];
    for (int i = 0; i < k - 1; i++) {
        queue.offer(arr[i]);
        result[i] = -1;
    }

    queue.offer(arr[k - 1]);
    result[k - 1] = queue.peek();

    for (int i = k; i < arr.length; i++) {
        if (arr[i] > queue.peek()) {
            queue.poll();
            queue.offer(arr[i]);
        }

        result[i] = queue.peek();
    }

    return result;
}

System.out.println(Arrays.toString(kthLargestInRange(new int[]{7, 8, 2, 5, 9, 1, 6}, 4)));

[-1, -1, -1, 2, 5, 5, 6]
